In [8]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import time


In [9]:
BASE = "https://books.toscrape.com/"

def parse_rating(rating_str):
    # rating as word -> number
    mapping = {"One":1,"Two":2,"Three":3,"Four":4,"Five":5}
    return mapping.get(rating_str, None)

titles, prices, ratings, availability, urls = [], [], [], [], []

page = "index.html"
while True:
    resp = requests.get(urljoin(BASE, page))
    soup = BeautifulSoup(resp.text, "html.parser")

    for book in soup.select(".product_pod"):
        title = book.h3.a["title"].strip()
        price_raw = book.select_one(".price_color").text.strip()   # e.g. '£53.74'
        price = price_raw.replace("£","")
        rating_word = book.p.get("class")[1]  # e.g. 'Three'
        rating = parse_rating(rating_word)
        avail = book.select_one(".availability").text.strip()
        rel = book.h3.a["href"]
        product_url = urljoin(BASE, rel)

        titles.append(title)
        prices.append(price)
        ratings.append(rating)
        availability.append(avail)
        urls.append(product_url)

    # next page
    next_btn = soup.select_one(".next a")
    if next_btn:
        page = next_btn['href']
        time.sleep(0.5)  # polite delay
    else:
        break

In [11]:
df = pd.DataFrame({
    "title": titles,
    "price_gbp": prices,
    "rating": ratings,
    "availability": availability,
    "product_page_url": urls
})
df.to_csv("books_data.csv", index=False)
print("Saved data/books_data.csv (rows: {})".format(len(df)))
print(df.head())

Saved data/books_data.csv (rows: 40)
                                   title price_gbp  rating availability  \
0                   A Light in the Attic    Â51.77       3     In stock   
1                     Tipping the Velvet    Â53.74       1     In stock   
2                             Soumission    Â50.10       1     In stock   
3                          Sharp Objects    Â47.82       4     In stock   
4  Sapiens: A Brief History of Humankind    Â54.23       5     In stock   

                                    product_page_url  
0  https://books.toscrape.com/catalogue/a-light-i...  
1  https://books.toscrape.com/catalogue/tipping-t...  
2  https://books.toscrape.com/catalogue/soumissio...  
3  https://books.toscrape.com/catalogue/sharp-obj...  
4  https://books.toscrape.com/catalogue/sapiens-a...  
